In [8]:
from os import path
import sys
sys.path.append(path.abspath(path.join(path.dirname(path.abspath('')), '.')))

from scripts.data_cleaning import load_csv, univariate_analysis, bivariate_analysis,correlation_matrix
from scripts.data_cleaning import clean_dataframe, analyze_and_remove_duplicates, analyze_categorical_features


In [9]:
print ("Loading data...",)
creditcard_data = load_csv('../Data/creditcard.csv')
fraud_data = load_csv('../Data/fraud_data_cleaned.csv')


Loading data...


2025-02-17 22:21:58,073 - INFO - ✅  CSV file '../Data/creditcard.csv' loaded successfully.
2025-02-17 22:21:58,233 - INFO - ✅  CSV file '../Data/fraud_data_cleaned.csv' loaded successfully.


In [30]:
import mlflow
from mlflow.sklearn import log_model
# Set the tracking URI (local or remote)
mlflow.set_tracking_uri("../mlruns")  # Default is ./mlruns

# Create or set an experiment
experiment_name = "Fraud Detection Experiment"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='/home/adane/Repository/Fraud-detection-for-e-commerce/notebook/../mlruns/781522624226821569', creation_time=1739813532949, experiment_id='781522624226821569', last_update_time=1739813532949, lifecycle_stage='active', name='Fraud Detection Experiment', tags={}>

In [33]:
from mlflow.models.signature import infer_signature
def train_and_evaluate(model, X_train, X_test, y_train, y_test, run_name):
    with mlflow.start_run(run_name=run_name):
        # Log parameters
        mlflow.log_param("model", type(model).__name__)
        
        # Train the model
        model.fit(X_train, y_train)
        # Infer the signature
        signature = infer_signature(X_train, y_train)

        # Create an input example
        input_example = X_train.iloc[:5]
        # Predict on test data
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        
        # Log the model
        mlflow.sklearn.log_model(model, "model",signature=signature,
        input_example=input_example)

In [19]:
# Separate features and target for creditcard.csv
X_creditcard = creditcard_data.drop(columns=['Class'])
y_creditcard = creditcard_data['Class']

# Separate features and target for Fraud_Data.csv
X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']

In [13]:
from sklearn.model_selection import train_test_split

# Split for creditcard.csv
X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(
    X_creditcard, y_creditcard, test_size=0.2, random_state=42, stratify=y_creditcard
)

# Split for Fraud_Data.csv
X_train_fd, X_test_fd, y_train_fd, y_test_fd = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize and train the model for creditcard.csv
lr_cc = LogisticRegression(max_iter=1000, random_state=42)
lr_cc.fit(X_train_cc, y_train_cc)

# Predict and evaluate
y_pred_cc_lr = lr_cc.predict(X_test_cc)
print("Logistic Regression (Credit Card):")
train_and_evaluate(lr_cc, X_train_cc, X_test_cc, y_train_cc, y_test_cc, "Logistic Regression creditcard")

/home/adane/Repository/Fraud-detection-for-e-commerce/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression (Credit Card):


/home/adane/Repository/Fraud-detection-for-e-commerce/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/adane/Repository/Fraud-detection-for-e-commerce/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema base

In [34]:

# Repeat for Fraud_Data.csv
lr_fd = LogisticRegression(max_iter=1000, random_state=42,class_weight='balanced')
lr_fd.fit(X_train_fd, y_train_fd)
y_pred_fd_lr = lr_fd.predict(X_test_fd)
print("\nLogistic Regression (Fraud Data):")
train_and_evaluate(lr_fd, X_train_fd, X_test_fd, y_train_fd, y_test_fd, "Logistic Regression fraud_data")



Logistic Regression (Fraud Data):


/home/adane/Repository/Fraud-detection-for-e-commerce/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [36]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model for creditcard.csv
rf_cc = RandomForestClassifier(n_estimators=100, random_state=42)
rf_cc.fit(X_train_cc, y_train_cc)

# Predict and evaluate
y_pred_cc_rf = rf_cc.predict(X_test_cc)
train_and_evaluate(rf_cc, X_train_cc, X_test_cc, y_train_cc, y_test_cc, "Random Forest creditcard")

print("Random Forest (Credit Card):")


# Repeat for Fraud_Data.csv
rf_fd = RandomForestClassifier(n_estimators=100, random_state=42)
rf_fd.fit(X_train_fd, y_train_fd)
y_pred_fd_rf = rf_fd.predict(X_test_fd)
print("\nRandom Forest (Fraud Data):")
train_and_evaluate(rf_fd, X_train_fd, X_test_fd, y_train_fd, y_test_fd, "Random Forest fraud_data")


/home/adane/Repository/Fraud-detection-for-e-commerce/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Random Forest (Credit Card):

Random Forest (Fraud Data):


/home/adane/Repository/Fraud-detection-for-e-commerce/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
